In [57]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import f1_score

In [2]:
data = pd.read_table("smsspamcollection/SMSSpamCollection", header=None, names=['label', 'text'])

In [3]:
data.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
data['label'] = data['label'].apply(lambda x: int(x == 'spam'))

In [5]:
data.head(10)

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [29]:
X_texts = data['text'].values
y = data['label'].values

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_texts)

In [8]:
scores = cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')

In [9]:
print(np.average(scores))

0.932640298361


In [10]:
X_test_texts = np.array(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                         "FreeMsg: Txt: claim your reward of 3 hours talk time",
                         "Have you visited the last lecture on physics?",
                         "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                         "Only 99$"])
X_test = vectorizer.transform(X_test_texts)

In [11]:
model = LogisticRegression()
model = model.fit(X, y)
y_pred = model.predict(X_test)

In [21]:
print(" ".join(map(str, y_pred)))

1 1 0 0 0


In [13]:
cross_val_scores = []

In [14]:
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    X = vectorizer.fit_transform(X_texts)
    cross_val_scores.append(np.average(cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')))

In [22]:
print(" ".join(map(lambda x: str(round(x, 2)), cross_val_scores)))

0.82 0.73 0.93


In [23]:
cross_val_scores_nb = []

In [24]:
for ngram_range in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    X = vectorizer.fit_transform(X_texts)
    cross_val_scores_nb.append(np.average(cross_val_score(MultinomialNB(), X, y, cv=10, scoring='f1')))

In [25]:
print(" ".join(map(lambda x: str(round(x, 2)), cross_val_scores_nb)))

0.65 0.38 0.89


In [19]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(X_texts)
print(np.average(cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')))

0.852859955417


In [20]:
vectorizer = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(X_texts)
print(np.average(cross_val_score(LogisticRegression(), X, y, cv=10, scoring='f1')))

0.932640298361


Видно, что качество с TfidfVectorizer тут хуже, чем с CountVectorizer

Попробуем подобрать параметры логистической регрессии:

In [92]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_texts)

In [93]:
def scorer(estimator, X, y):
    return f1_score(estimator.predict(X), y)

In [54]:
grid = {'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None],
        'C': [1e-3, 0.01, 0.1, 0.5, 1]}
search = GridSearchCV(LogisticRegression(), param_grid=grid, scoring=scorer, n_jobs=8, cv=5)
search = search.fit(X, y)

/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricW

In [55]:
search.best_estimator_

LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [56]:
search.best_score_

0.94273666197945516

Теперь попробуем SGDClassifier:

In [94]:
grid = {'penalty': ['l1', 'l2', 'elasticnet', 'none'], 'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 0.1],
        'class_weight': ['balanced', None], 'l1_ratio': [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5]}
search = GridSearchCV(SGDClassifier(), param_grid=grid, scoring=scorer, n_jobs=8, cv=5)
search = search.fit(X, y)

/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/nastya/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricW

In [95]:
search.best_estimator_

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.3, learning_rate='optimal',
       loss='hinge', n_iter=5, n_jobs=1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [96]:
search.best_score_

0.94390896786616474

Итак, у логистической регрессии лучшая f1_score 0.9427, а у SGDClassifier 0.9439.

### Выводы:
* Если датасет несбалансированный, то модель, не учитывающая это, работает хуже.

* По маленькой фразе нельзя сказать, принадлежит она к спаму или нет.

* Логистическая регрессия работает хорошо, но учет n-грамм не улучшил качества, возможно это из-за недостатка данных.

* Возможно, если бы датасет был больше, то наивный Байес с 1,2,3-граммами сработал бы лучше.

* Странно, что TfidfVectorizer фичи сработали хуже CountVectorizer.